In [1]:
import pandas as pd
import numpy as np
from gensim import corpora, models, similarities
from gensim.test.utils import get_tmpfile
from collections import defaultdict
import random

C:\Users\g2018\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
!pip install gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.23
    Uninstalling Cython-0.29.23:
      Successfully uninstalled Cython-0.29.23


# Import Data

In [28]:
# response_path = "./data/ABA_train.csv"
response_path = "./data/new_data.csv"
candidate_path = "./data/嘉音讯_重庆联通_未来会员STT入库表.xlsx"

In [29]:
response_data

,Context,Response,Label,SessionId
index,,,,
1,你好，中国联通从204手机尾号3682的机主吗？-------嗯对。,好抱歉打扰到您了，感谢您的接听，这边是重庆联通给你来电的，通知到您从这个月开始，每幺十五元给...,1,1
2,好抱歉打扰到您了，感谢您的接听，这边是重庆联通给你来电的，通知到您从这个月开始，每幺十五元给...,好了，这边跟您说一下，咱们领取每要15元的暂时的会员，您按照短信步骤在微信上领取就行了，话费...,1,1
3,好了，这边跟您说一下，咱们领取每要15元的暂时的会员，您按照短信步骤在微信上领取就行了，话费...,嗯这边耽误您30秒时间就可以了先生。~~~然后的话啊首月是征到第二天到账的，后期是10号前到...,1,1
4,嗯这边耽误您30秒时间就可以了先生。~~~然后的话啊首月是征到第二天到账的，后期是10号前到...,好的，打扰到您了，祝您生活愉快，再见。,1,1
5,请问是重庆通给您来电，请问您现在今天这手机尾号2919是您在用的对吗？-------对。~~~啊,你好先生，来电视短信资源通知时，这个月开始每月25元，您可以领取到两款VIP，扣押了~~~比...,1,2
...,...,...,...,...
28659,您好，中国联通建议来电，请问本机尾号1202是您在用的对吧？-------两个，一个是谁？,啊很抱歉打扰到您。~~~来电是感谢您对联通的支持，从这个月开始啊每个月25元给您领取两款VI...,1,7450
28660,啊很抱歉打扰到您。~~~来电是感谢您对联通的支持，从这个月开始啊每个月25元给您领取两款VI...,啊就是说从这个月开始，嘛每个月25块钱，你可以领取到2块5VIP会员，嘛~~~有20多款热门...,1,7450
28661,啊就是说从这个月开始，嘛每个月25块钱，你可以领取到2块5VIP会员，嘛~~~有20多款热门...,需要每个月25块钱的，都每个月25块钱马上给您领取到两款VIP会员了。~~~那这边也是留意到。,1,7450


In [30]:
response_data = pd.read_csv(response_path,index_col=0)
standard_data = pd.read_excel(candidate_path,index_col=0)

In [31]:
df = response_data
df2 = standard_data

In [32]:
df = df.rename(columns={'Context':'context', 'Response':'response', "Label":'label_'})
df.head()

,context,response,label_,SessionId
index,,,,
1,你好，中国联通从204手机尾号3682的机主吗？-------嗯对。,好抱歉打扰到您了，感谢您的接听，这边是重庆联通给你来电的，通知到您从这个月开始，每幺十五元给...,1,1
2,好抱歉打扰到您了，感谢您的接听，这边是重庆联通给你来电的，通知到您从这个月开始，每幺十五元给...,好了，这边跟您说一下，咱们领取每要15元的暂时的会员，您按照短信步骤在微信上领取就行了，话费...,1,1
3,好了，这边跟您说一下，咱们领取每要15元的暂时的会员，您按照短信步骤在微信上领取就行了，话费...,嗯这边耽误您30秒时间就可以了先生。~~~然后的话啊首月是征到第二天到账的，后期是10号前到...,1,1
4,嗯这边耽误您30秒时间就可以了先生。~~~然后的话啊首月是征到第二天到账的，后期是10号前到...,好的，打扰到您了，祝您生活愉快，再见。,1,1
5,请问是重庆通给您来电，请问您现在今天这手机尾号2919是您在用的对吗？-------对。~~~啊,你好先生，来电视短信资源通知时，这个月开始每月25元，您可以领取到两款VIP，扣押了~~~比...,1,2


In [33]:
standard_data.head()

,是否重点句,重点句标题,客户回复分类,疑问句分类,文案,簇心编码
客户状态,,,,,,
1,是,问候句,NaN,NaN,您好，这边是联通客服来电的，请问手机尾号****是你在用的对吧？,1000
2,是,质检句1,NaN,NaN,来电是感谢您对联通的支持。\n您从下个月开始，每月只要25元就可以领取两个VIP会员。\n因...,1001
2,否,质检句1,Bq,我需要交多25块钱吗？,就每个月25元，这边有话费40元返还给您抵扣月租的，分四月每月到账10元，希望可以帮您几个月...,1002
2,否,质检句1,Bq,我的套餐会改变吗？,不会影响您的套餐的，每月25元，联通还会返还40元话费给您，分四月每月到账10元，希望可以帮...,1003
2,否,质检句1,Bq,我已经有会员了,没关系啊，这边也是可以给您续的，而且我们这个是有二十多种会员，您每个月可以任意选两个用，每个...,1004


jieba中文分词

In [7]:
import jieba

In [34]:
#  去停用词
#  加载停用词列表
def load_stopword(filepath):
    f_stop = open(filepath, encoding='utf-8') # 中文停用词表
    sw = [line.strip() for line in f_stop] # strip() 方法用于移除字符串头尾制定的字符(默认为空格)
    f_stop.close() 
    return sw

# jieba中文分词，去停用词
def seg_word(sentence):
    sentence_seged = jieba.lcut(sentence.strip(), use_paddle = True)
    stopwords = load_stopword('./data/new_stop_words.txt')
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords:
            if word != '\t':
                outstr += word 
                outstr += " "
    # print(outstr)
    return outstr

In [35]:
after = [] 
for sentence in df2["文案"]:
  after.append(seg_word(sentence))
df2["文案"] = after
df2["文案"]

客户状态
1           您好 这边 是 联通 客服 来电 的 请问 手机 尾号 是 你 在 用 的 对 吧 
2    来电 是 感谢您 对 联通 的 支持 \n 您 从 下个月 开始 每月 只要 25 元 就 ...
2    就 每个 月 25 元 这边 有 话费 40 元 返还 给 您 抵扣 月租 的 分 四月 每...
2    不会 影响 您 的 套餐 的 每月 25 元 联通 还会 返还 40 元 话费 给 您 分 ...
2    没关系 啊 这边 也 是 可以 给 您 续 的 而且 我们 这个 是 有 二十多种 会员 您...
2    就 耽误 您 一两分钟 时间 现在 有 会员 给 到 您 用 联通 还会 返还 40 元 话...
3    好 的 感谢您 的 支持 跟 你 说 一下 会员 怎么 领取 您 根据 短信 在 微信 公众...
3    就 每个 月 25 元 这边 有 话费 40 元 返还 给 您 抵扣 月租 的 分 四月 每...
3    不会 影响 您 的 套餐 的 每月 25 元 联通 还会 返还 40 元 话费 给 您 分 ...
3    没关系 啊 这边 也 是 可以 给 您 续 的 而且 我们 这个 是 有 二十多种 会员 您...
3    就 耽误 您 一两分钟 时间 现在 有 会员 给 到 您 用 联通 还会 返还 40 元 话...
4    好 的 使用 中 您 可以 下载 中国联通 APP 查询 您 的 消费 情况 关注 最新 活...
Name: 文案, dtype: object

In [36]:
after2 = []
after3 = []
for st in zip(df["context"],df["response"]):
  after2.append(seg_word(st[0]))
  after3.append(seg_word(st[1]))
df["context"] = after2
df["response"] = after3
print(df.head())

                                                 context  \
index                                                      
1           你好 中国联通 从 204 手机 尾号 3682 的 机主 吗 ------- 嗯 对    
2      好 抱歉 打扰到 您 了 感谢您 的 接听 这边 是 重庆 联通 给 你 来电 的 通知 到...   
3      好 了 这边 跟 您 说 一下 咱们 领取 每要 15 元 的 暂时 的 会员 您 按照 短...   
4      嗯 这边 耽误 您 30 秒 时间 就 可以 了 先生 然后 的话 啊 首 月 是 征到 第...   
5      请问 是 重庆 通给 您 来电 请问 您 现在 今天 这 手机 尾号 2919 是 您 在 ...   

                                                response  label_  SessionId  
index                                                                        
1      好 抱歉 打扰到 您 了 感谢您 的 接听 这边 是 重庆 联通 给 你 来电 的 通知 到...       1          1  
2      好 了 这边 跟 您 说 一下 咱们 领取 每要 15 元 的 暂时 的 会员 您 按照 短...       1          1  
3      嗯 这边 耽误 您 30 秒 时间 就 可以 了 先生 然后 的话 啊 首 月 是 征到 第...       1          1  
4                               好 的 打扰到 您 了 祝您 生活 愉快 再见        1          1  
5      你好 先生 来 电视 短信 资源 通知 时 这个 月 开始 每月 25 元 您 可以 领取 ...       1          2  


In [37]:
response = [i.split() for i in df.response]
standard = [i.split() for i in df2["文案"]] 

,context,response,label_,SessionId,standard
index,,,,,
1,你好 中国联通 从 204 手机 尾号 3682 的 机主 吗 ------- 嗯 对,好 抱歉 打扰到 您 了 感谢您 的 接听 这边 是 重庆 联通 给 你 来电 的 通知 到...,1,1,来电 是 感谢您 对 联通 的 支持 \n 您 从 下个月 开始 每月 只要 25 元 就 ...
2,好 抱歉 打扰到 您 了 感谢您 的 接听 这边 是 重庆 联通 给 你 来电 的 通知 到...,好 了 这边 跟 您 说 一下 咱们 领取 每要 15 元 的 暂时 的 会员 您 按照 短...,1,1,好 的 感谢您 的 支持 跟 你 说 一下 会员 怎么 领取 您 根据 短信 在 微信 公众...
3,好 了 这边 跟 您 说 一下 咱们 领取 每要 15 元 的 暂时 的 会员 您 按照 短...,嗯 这边 耽误 您 30 秒 时间 就 可以 了 先生 然后 的话 啊 首 月 是 征到 第...,1,1,好 的 感谢您 的 支持 跟 你 说 一下 会员 怎么 领取 您 根据 短信 在 微信 公众...
4,嗯 这边 耽误 您 30 秒 时间 就 可以 了 先生 然后 的话 啊 首 月 是 征到 第...,好 的 打扰到 您 了 祝您 生活 愉快 再见,1,1,好 的 使用 中 您 可以 下载 中国联通 APP 查询 您 的 消费 情况 关注 最新 活...
5,请问 是 重庆 通给 您 来电 请问 您 现在 今天 这 手机 尾号 2919 是 您 在 ...,你好 先生 来 电视 短信 资源 通知 时 这个 月 开始 每月 25 元 您 可以 领取 ...,1,2,来电 是 感谢您 对 联通 的 支持 \n 您 从 下个月 开始 每月 只要 25 元 就 ...


In [39]:
df2.head()

,是否重点句,重点句标题,客户回复分类,疑问句分类,文案,簇心编码
客户状态,,,,,,
1,是,问候句,NaN,NaN,您好 这边 是 联通 客服 来电 的 请问 手机 尾号 是 你 在 用 的 对 吧,1000
2,是,质检句1,NaN,NaN,来电 是 感谢您 对 联通 的 支持 \n 您 从 下个月 开始 每月 只要 25 元 就 ...,1001
2,否,质检句1,Bq,我需要交多25块钱吗？,就 每个 月 25 元 这边 有 话费 40 元 返还 给 您 抵扣 月租 的 分 四月 每...,1002
2,否,质检句1,Bq,我的套餐会改变吗？,不会 影响 您 的 套餐 的 每月 25 元 联通 还会 返还 40 元 话费 给 您 分 ...,1003
2,否,质检句1,Bq,我已经有会员了,没关系 啊 这边 也 是 可以 给 您 续 的 而且 我们 这个 是 有 二十多种 会员 您...,1004


# TFIDF

In [40]:
def get_response(query, responses):    

    dictionary = corpora.Dictionary(responses)
    #print(dictionary.token2id)  
    
    # 建立语料库
    corpus = [dictionary.doc2bow(response) for response in responses]
    
    # 初始化一个tfidf模型,可以用它来转换向量（词袋整数计数），表示方法为新的表示方法（Tfidf 实数权重）
    tfidf = models.TfidfModel(corpus)
    
    # 将整个语料库转为tfidf表示方法
    corpus_tfidf = tfidf[corpus]

    # 使用doc2bow方法对每个不同单词的词频进行了统计，并将单词转换为其编号，然后以稀疏向量的形式返回结果
    new_vec = dictionary.doc2bow(query)
    new_vec_tfidf = tfidf[new_vec] # 将待比较文档转换为tfidf表示方法
   

    # 使用上一步得到的带有tfidf值的语料库建立索引
    index = similarities.MatrixSimilarity(corpus_tfidf)


    # 计算要比较的文档与语料库中每篇文档的相似度
    sims = index[new_vec_tfidf]
    sims_list = sims.tolist()
    
    return sims_list

# **get standard**

In [42]:
list1 = []
for i in range(len(response)):
  index = np.argmax(get_response(response[i],standard))
  list1.append(df2["文案"].values[index])

df["standard"] = list1
df.head()
df.to_csv("standard.csv")

In [12]:
print(df.response[5])
print('')
print(df.standard[5])

嗯 那 先生 你 到时候 记得 根据 短信 在 微信 公众 号 上面 关于 音乐 约定 会员 就 可以 用 的 了 好 吧 

好 的 感谢您 的 支持 跟 你 说 一下 会员 怎么 领取 您 根据 短信 在 微信 公众 号 沃 音乐 领取 就 好 了 
 每月 25 元除 月租 外 一次性 扣取 立即 生效 自动 续订 
 联通 还会 返还 您 40 元 话费 分 4 个 月 每月 返还 10 元 话费 会 在 下个月 10 号 前 到 账 
 给 您 受理 好 收到 生效 短信 后 就 可以 放心 用 的 了 好 吧 


In [133]:
df.to_csv("./data/tfidf.csv")